In [1]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import glob

from skimage.feature import corner_harris, corner_peaks
import cv2

In [2]:
def animate_tool(frames, interval, repeat, repeat_delay):
    fig = plt.figure()
    im = plt.imshow(frames[0], cmap=plt.get_cmap('gray'), vmin=0, vmax=255, animated=True)
    
    def updatefig(j):
        im.set_array(frames[j])
        return im,

    ani = animation.FuncAnimation(fig, updatefig, frames=range(len(frames)), blit=True,
                                  interval=interval, repeat=repeat, repeat_delay=repeat_delay)
    return ani

In [49]:
if __name__ == '__main__':
    path = 'Images/Q1'
    imagelist = []
    
    for filename in glob.glob(os.path.join(path, '*.bmp')): 
        im = plt.imread(filename)
        fn = filename.split('/')[-1].split('.')[0]
        imagelist.append((fn, im))

    frames = [i[1] for i in imagelist]

    # kick off the animation
#     ani = animate_tool(frames, 100, True, 500)
#     mins = []
#     for frame in frames:
#         ch = corner_harris(frame, k=0.04)
#         mins.append(np.min(ch))
#     print np.min(mins)
#     print np.min(ch)
#     peaks = corner_peaks(ch, min_distance=1, threshold_abs=np.min(ch))
#     print peaks.shape

    img = cv2.imread('Images/Q1/car1.bmp')
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    ch = corner_harris(gray, k=0.04)
    peaks = corner_peaks(ch, min_distance=1, threshold)
    print peaks.shape

#     print imagelist[0][0]

#     gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,7,3,0.04)
    print np.min(dst)
    peaks = corner_peaks(dst, min_distance=1)
    print peaks.shape

(83, 2)
-0.00177722
(120, 2)
